<a href="https://colab.research.google.com/github/Sri-hari-vardhan/Fmml-labs/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
The size of the validation set can significantly impact the accuracy of the validation set itself and, indirectly, the overall machine learning model. Here's how it's affected when you increase or reduce the percentage of the validation set:

1. **Increase the Percentage of Validation Set:**
   - Pros: A larger validation set can provide a more reliable estimate of the model's performance. It allows for better assessment of generalization and helps in identifying overfitting or underfitting.
   - Cons: With a larger validation set, you have fewer data points for training, which can result in a model that doesn't learn as well from the available data. This can lead to a higher bias in the model.

2. **Reduce the Percentage of Validation Set:**
   - Pros: A smaller validation set means more data is available for training, which can result in a model that learns better from the data. This can lead to lower bias in the model.
   - Cons: With a smaller validation set, the estimate of the model's performance might be less reliable. There's a higher chance of variance in the performance estimate due to the limited validation data.

The key is to strike a balance. A common practice is to split the dataset into training, validation, and test sets, where the training set is the largest, followed by the validation set, and the test set is smaller. The validation set's size depends on the overall dataset size and the specific problem, but common splits might be something like 70% for training, 15% for validation, and 15% for testing.

Ultimately, the choice of the percentage of the validation set depends on the dataset's size, your computational resources, and the need for reliable model evaluation. It's often a trade-off between having a good estimate of model performance and ensuring the model learns effectively from the data.
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
The size of the training and validation sets can significantly affect how well you can predict the accuracy on the test set using the validation set. Here's how:

1. **Large Training Set and Small Validation Set:**
   - Pros: A large training set allows the model to learn complex patterns from the data, which can result in better performance. However, a small validation set may still provide a reasonable estimate of model performance.
   - Cons: With a small validation set, the estimate of model performance may have higher variance. It might not capture the model's generalization behavior accurately, leading to less confidence in predicting test set accuracy.

2. **Small Training Set and Large Validation Set:**
   - Pros: A large validation set can provide a more stable estimate of model performance, which can be closer to the true test set performance. It's particularly useful when you have limited training data.
   - Cons: A small training set might result in a model that doesn't learn well from the data, leading to higher bias in the model. The model's performance on the validation set might not accurately reflect how it will perform on unseen data.

3. **Balanced Training and Validation Set Sizes:**
   - Pros: A balanced approach, where both training and validation sets are of reasonable size, often strikes a good balance between model learning and performance estimation. It provides a more reliable estimate of how the model will perform on the test set.
   - Cons: It might require a larger overall dataset, and you may need to carefully allocate data points to both sets.

In practice, it's common to use techniques like cross-validation (e.g., k-fold cross-validation) to mitigate the impact of the validation set's size. Cross-validation involves splitting the data into multiple folds, training and validating the model on different subsets in an iterative manner. This helps in obtaining a more robust estimate of model performance.

Ultimately, the choice of the training and validation set sizes depends on factors like the dataset's size, the complexity of the model, and the need for reliable performance estimation. A good balance is often preferred to obtain meaningful predictions of test set accuracy using the validation set.
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?
The choice of what percentage to reserve for the validation set depends on several factors, including the size of your dataset, the complexity of your model, and your computational resources. There is no one-size-fits-all answer, but a common practice is to use a split like 70-15-15 or 80-10-10 for training, validation, and testing, respectively. Here are some considerations to help you strike a balance:

1. **Dataset Size**: If you have a large dataset, you can afford to allocate a smaller percentage for validation and testing while still having sufficient data for training. In this case, something like 70% for training, 15% for validation, and 15% for testing could work well.

2. **Model Complexity**: More complex models often require larger validation sets to evaluate their performance accurately. If you're working with a simple model, you might get away with a smaller validation set.

3. **Computational Resources**: If you have limited computational resources and training a model is time-consuming, you might prefer a smaller validation set to speed up experimentation. Conversely, if resources are not a concern, you can allocate a larger validation set.

4. **Need for Reliable Estimates**: If you need highly reliable estimates of model performance and have sufficient data, a larger validation set can provide more confidence in the model's behavior.

5. **Cross-Validation**: Consider using techniques like k-fold cross-validation if you want to make the most of your data. Cross-validation helps in utilizing the available data more efficiently by repeatedly splitting it into training and validation sets.

In practice, it's often a matter of experimentation and validation set size can vary depending on the specific problem. Start with a reasonable split based on the guidelines above, and if you find that your model's performance estimates are unstable or not reliable, you can adjust the validation set size accordingly.

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
Yes, averaging the validation accuracy across multiple splits, such as in k-fold cross-validation, generally provides more consistent and reliable results compared to a single split. Here's why:

1. **Reduced Variance**: Averaging validation accuracy over multiple splits helps reduce the impact of random variations in the data. In a single split, the specific random selection of the validation set can lead to variations in the performance estimate. When you average over multiple splits, these variations tend to cancel out, providing a more stable and representative estimate of model performance.

2. **Better Generalization Estimate**: Cross-validation allows you to evaluate the model's performance on different subsets of the data. This gives you a better estimate of how well the model generalizes to unseen data, as it's tested on multiple "unseen" validation sets during each fold.

3. **Robustness to Data Imbalances**: In cases where the dataset is imbalanced, meaning some classes or conditions are underrepresented, cross-validation ensures that each class or condition gets a chance to be in the validation set during different folds, improving the overall assessment of model performance.

4. **More Efficient Use of Data**: Cross-validation efficiently utilizes the available data. Instead of reserving a fixed portion of the dataset for validation, it rotates through the data so that each data point is used for both training and validation at some point. This is especially beneficial when you have limited data.

In summary, averaging validation accuracy over multiple splits, as in cross-validation, provides a more robust and reliable estimate of a model's performance. It's a widely used technique to assess a model's ability to generalize and to obtain a more stable evaluation of model performance, especially when working with limited data or dealing with variations in the data.
2. Does it give more accurate estimate of test accuracy?
Averaging validation accuracy across multiple splits, as done in techniques like k-fold cross-validation, provides a more accurate estimate of how well a model is likely to perform on unseen data compared to a single validation split. However, it's essential to understand that this estimate is still an approximation and not a direct measure of test accuracy. Here's why:

1. **Closer to Reality**: Cross-validation simulates the process of training and testing a model on multiple different subsets of data. By doing so, it provides a more comprehensive evaluation of a model's generalization ability. This means that the estimate is closer to what you might expect on unseen data.

2. **Reduced Variance**: Cross-validation reduces the impact of random variations in the data that can affect a single validation split. Averaging over multiple splits helps to smooth out these variations, resulting in a more stable estimate of model performance.

3. **Robustness**: Cross-validation is robust to variations in the dataset, including potential data imbalances and outliers. It ensures that each data point participates in both training and validation during different folds, leading to a more representative evaluation.

4. **Efficient Data Utilization**: Cross-validation efficiently utilizes the available data, making it especially valuable when working with limited datasets.

However, it's important to note that the estimate obtained through cross-validation is still based on the same dataset, and it may not fully capture the variability and challenges that real-world unseen data might present. Therefore, while it provides a better estimate of model performance than a single validation split, the actual test accuracy on completely new, unseen data could still differ, especially if the test data has different characteristics or properties than the training and validation data.

In practice, cross-validation is a valuable tool for model evaluation, and it helps you make informed decisions about model selection, hyperparameter tuning, and generalization. It provides a more accurate estimate of how your model is likely to perform in the real world, but it's always important to validate your model on a separate, held-out test set when assessing its final performance.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
The number of iterations in machine learning, such as the number of iterations during training a model (e.g., epochs in deep learning), can have an impact on the estimate of model performance. However, whether a higher number of iterations results in a better estimate depends on several factors:

1. **Early Stopping**: Many machine learning practitioners use a technique called early stopping, where training is stopped when the model's performance on the validation set starts to degrade. In such cases, increasing the number of iterations beyond the point where early stopping occurs might not lead to better estimates. In fact, it can result in overfitting to the training data.

2. **Convergence**: The number of iterations required for a model to converge to its optimal performance can vary. Some models may converge quickly, while others may require more iterations to reach their best performance. Increasing the number of iterations can be beneficial if the model hasn't converged yet, but once it reaches convergence, additional iterations may have diminishing returns.

3. **Computational Resources**: Training models with a higher number of iterations can be computationally expensive and time-consuming. Depending on your computational resources and time constraints, you may need to balance the number of iterations with the available resources.

4. **Data Size**: The size of your dataset can also influence the impact of the number of iterations. Smaller datasets might benefit from more iterations to allow the model to learn more from the limited data, while larger datasets might not require as many iterations.

5. **Hyperparameters**: Other hyperparameters, such as learning rate and batch size, can interact with the number of iterations. The optimal number of iterations can depend on the values of these hyperparameters.

In summary, there isn't a one-size-fits-all answer to whether a higher number of iterations results in a better estimate of model performance. It depends on the specific problem, model architecture, dataset size, and other hyperparameters. It's often recommended to use techniques like early stopping and monitor validation performance to determine the appropriate number of iterations for your specific machine learning task.
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Increasing the number of iterations can help to some extent when dealing with a very small training dataset, but it has its limitations, and it may not fully compensate for the lack of data. Here are some considerations:

Pros of Increasing Iterations with a Small Training Dataset:

1. **Learning More from Limited Data**: With more iterations, the model has more opportunities to see the limited training data, which can help it capture some of the underlying patterns in the data.

2. **Potential for Better Local Optima**: Some machine learning algorithms may benefit from more iterations to explore a larger portion of the solution space, increasing the chances of finding a better local optima.

Cons and Limitations:

1. **Overfitting**: If you increase the number of iterations significantly with a very small training dataset, the model may start to overfit. It can begin to memorize the training data rather than learning meaningful generalizations, leading to poor performance on new, unseen data.

2. **Computational Cost**: Training with a high number of iterations can be computationally expensive, especially for complex models. It may not be feasible in terms of time and resources.

3. **Limited Data**: The fundamental limitation of a small training dataset remains. More iterations won't magically create more data, and the model's ability to generalize will still be constrained by the limited amount of information in the training set.

In practice, it's essential to strike a balance between the number of iterations and the available data. If you have a very small training dataset, consider techniques like data augmentation (if applicable), using simpler models, or applying regularization methods to prevent overfitting. Additionally, you can still utilize techniques like k-fold cross-validation to obtain a more robust estimate of model performance even with limited data.